In [1]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

In [209]:
x="https://gis.wilco.org/arcgis/rest/services/wcchd/Williamson_County_Cases/FeatureServer/0/query?f=json&where=Number_of_Cases%3E0&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&groupByFieldsForStatistics=City_of_Residence&outStatistics=%5B%7B%22statisticType%22%3A%22sum%22%2C%22onStatisticField%22%3A%22Number_of_Cases%22%2C%22outStatisticFieldName%22%3A%22value%22%7D%5D&outSR=102100"

In [210]:
x==constants.WILCO_CITY_XHR_URL

True

In [8]:
df = pd.read_html(constants.WILCO_URL, match="Summary of COVID")[0]

In [160]:
def find_separator_rows(df):
    """The HTML table has an empty row to separate logical tables.  Get the indices of those"""
    it = df.iterrows()
    # skip to "Summary" row
    for idx, row in it:
        if not row.isna().all() and row[0].startswith("Summary of COVID"):
            _, header_row = next(it)
            yield (idx+2, header_row[0])
            break

    # now look for separator rows of all NaNs
    for idx, row in it:
        if row.isna().all():
            _, header_row = next(it)
            
            yield (idx+2, header_row[0])

In [167]:
def find_separator_rows(df):
    """The HTML table has an empty row to separate logical tables.  Get the indices of those"""
    it = df.iterrows()
    # skip to "Summary" row
    for idx, row in it:
        if not row.isna().all() and row[0].startswith("Summary of COVID"):
            start_idx = idx + 2
            _, header_row = next(it)
            break

    # now look for separator rows of all NaNs
    for idx, row in it:
        if row.isna().all():
            yield (start_idx, idx, header_row[0])
            start_idx = idx + 2
            _, header_row = next(it)
    yield (start_idx, idx, header_row[0])

In [182]:
def generate_subtables(df):
    common_headers = ['Number of Cases', '% of Total Cases']
    for start_idx, end_idx, categ in find_separator_rows(df):
        subdf = df.copy().iloc[start_idx:end_idx]
        subdf.columns = [categ] + common_headers
        yield subdf.set_index(categ)


In [183]:
subtables = list(generate_subtables(df))

In [193]:
city_data = subtables[0]
gender_data = subtables[1]
age_data = subtables[2]
status_data = subtables[3]

In [194]:
today = date.today().isoformat()

In [196]:
city_data.to_csv(f"{constants.DATA_PATH}/wilco/bycity/{today}.csv", header=True, index=True)
gender_data.to_csv(f"{constants.DATA_PATH}/wilco/bygender/{today}.csv", header=True, index=True)
age_data.to_csv(f"{constants.DATA_PATH}/wilco/byage/{today}.csv", header=True, index=True)
status_data.to_csv(f"{constants.DATA_PATH}/wilco/bystatus/{today}.csv", header=True, index=True)



Append today's data to timeseries

In [201]:
REPORTS = [
    ("City of Residence", "bycity", city_data),
    ("Gender", "bygender", gender_data),
    ("Age Group", "byage", age_data),
    ("Status", "bystatus", status_data)
]

In [206]:
for report in REPORTS:
    df_timeseries = pd.read_csv(f"{constants.DATA_PATH}/wilco/{report[1]}/timeseries.csv").set_index(report[0])
    df = report[2][["Number of Cases"]].rename(columns={"Number of Cases": today})
    if today in df_timeseries.columns:
        df_timeseries.drop(columns=today, inplace=True)
    df_timeseries = pd.concat([df_timeseries, df], axis=1).rename_axis(report[0])
    df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)
    df_timeseries.to_csv(f"{constants.DATA_PATH}/wilco/{report[1]}/timeseries.csv", header=True, index=True)